In [ ]:
'''
@author: osamah Abdelhaq last edit: 6/29/2020
'''
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
import os
import json
import pandas as pd
from getpass import getpass

# Facebook Crawler

Set to scrape information from about USPS's 80 most recent posts, FLDEO's 50 most recent posts, and IRS's 106 most recent posts.

Uses provided mbasic facebook link to a page to scrape posts, comments, replies, likes on comments and replies, and timestamps.

In [ ]:
class FacebookBot(webdriver.PhantomJS):
    """Class for browsing facebook"""

    def __init__(self):
        
        self.browser = webdriver.Chrome(executable_path="C:\chromedriver_win32\chromedriver.exe")

    def login(self, email, password):
        '''Log to facebook using email and password'''
        url = "https://mbasic.facebook.com"
        self.browser.get(url)
        email_element = self.browser.find_element_by_name("email")
        email_element.send_keys(email)
        pass_element = self.browser.find_element_by_name("pass")
        pass_element.send_keys(password)
        pass_element.send_keys(Keys.ENTER)
        if self.browser.find_element_by_class_name("bi"):
            self.browser.find_element_by_class_name("bp").click();
        try:
            self.browser.find_element_by_name("xc_message")
            print("Logged in")
            return True
        except NoSuchElementException as e:
            print("Fail to login")
            return False
                
    def getContent(self, url, deep):
        '''
        Get a list of comments (list:Post) in post url(str) iterating deep(int) times in the page
        
        Collects the post content and likes, followed by all comments including user name, timestamp, and likes.
        
        After collecting comments on that page, it collects all reply threads with more than one reply.
        
        Then it finds more comments on that post until no comments remain.
        
        Repeats for deep (int) amount of posts. 6 deep equals 24 posts.
        
        Note: unicode is scraped, some comments are repeated when involved in reply conversation.
        '''

        self.browser.get(url)
        
        content = []
        
        #gathers links to the posts
        links = []
        for i in range(deep):
            for a in self.browser.find_elements_by_xpath('.//a'):
                try:
                    l = a.get_attribute('href')
                    
                    #****change to match source code link
                    if ('/IRS/photos/a.' in l):
                        if (l[-4:] == 'AW-R'):
                            links.append(l)
                except:
                    continue
            self.browser.find_element_by_partial_link_text('Show more').click()
            time.sleep(1)
            
        #eliminate link duplicates        
        links = set(links)
        
        print('Number of posts being scraped: ', len(links))
        
        for link in links:
            time.sleep(1)
            self.browser.get(link)
            time.sleep(2)
            
            #*****change class name to match that of the page source code
            post = self.browser.find_element_by_class_name('msg') #get post content
            time.sleep(1)
            content.append(post.text)

            for i in range(20):
                try:
                    x = self.browser.find_element_by_partial_link_text('View more').get_attribute('href') #gets link to more comments
                except:
                    break
                else:
                    #*****change class name to match that of the page source code
                    comm = self.browser.find_elements_by_class_name('_55wr') #get first level comments

                    for t in comm:
                        content.append(t.text)

                    r = self.browser.find_elements_by_partial_link_text('replies') #only collects if more than one reply, gathers likes, date posted

                    reply_links = []

                    for m in r:
                        time.sleep(1)
                        reply_links.append(m.get_attribute('href')) #get all reply conversations

                    for lin in reply_links:
                        time.sleep(1)
                        self.browser.get(lin)
                        #*****change class name to get replies
                        rep = self.browser.find_elements_by_class_name('bd') #get the replies content

                        for e in rep:
                            content.append(e.text)
                    
                    time.sleep(1)
                    self.browser.get(x) #get more comments 
                    time.sleep(1)

        self.browser.close()
        
        return content

In [ ]:
#====================MAIN===================
def main():
#====================Login====================
    username = input("Username: ") 
    password = getpass("password: ")
    bot=FacebookBot()
    bot.login(username, password)
    
#====================Get Content from page======================

    #must pass an mbasic facebook url 
    comments = bot.getContent("https://mbasic.facebook.com/IRS/?refid=17&_ft_=mf_story_key.10157813581394735%3Atop_level_post_id.10157813581394735%3Atl_objid.10157813581394735%3Acontent_owner_id_new.247781869734%3Athrowback_story_fbid.10157813581394735%3Apage_id.247781869734%3Aphoto_id.10157813581394735%3Astory_location.4%3Astory_attachment_style.photo%3Atds_flgs.3%3Apage_insights.%7B%22247781869734%22%3A%7B%22page_id%22%3A247781869734%2C%22page_id_type%22%3A%22page%22%2C%22actor_id%22%3A247781869734%2C%22dm%22%3A%7B%22isShare%22%3A0%2C%22originalPostOwnerID%22%3A0%7D%2C%22psn%22%3A%22EntPhotoNodeBasedEdgeStory%22%2C%22post_context%22%3A%7B%22object_fbtype%22%3A22%2C%22publish_time%22%3A1590937243%2C%22story_name%22%3A%22EntPhotoNodeBasedEdgeStory%22%2C%22story_fbid%22%3A%5B10157813581394735%5D%7D%2C%22role%22%3A1%2C%22sl%22%3A4%2C%22targets%22%3A%5B%7B%22actor_id%22%3A247781869734%2C%22page_id%22%3A247781869734%2C%22post_id%22%3A10157813581394735%2C%22role%22%3A1%2C%22share_id%22%3A0%7D%5D%7D%7D%3Athid.247781869734%3A306061129499414%3A69%3A0%3A1593586799%3A6104849151741984672&__tn__=C-R"\
                                       ,deep=30)
    
    #*****Change Storage path if preferred*****
    # save the data
    with open('Facebook_IRS_Content.json', 'w') as out_f:
        json.dump(comments, out_f)
        
if __name__=="__main__":
    main()